<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/1001_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
from torch import nn 

In [ ]:
class MLPLayer(nn.Module):
    """
    Head for getting sentence representations over RoBERTa/BERT's CLS representation.
    """

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, features, **kwargs):
        x = self.dense(features)
        x = self.activation(x)

        return x

In [ ]:
class BatchNormWrapper(nn.Module):
    def __init__(self, m):
        super(BatchNormWrapper, self).__init__()
        self.m = m
        self.m.eval()  # Set the batch norm to eval mode

    def forward(self, x):
        input_type = x.dtype
        x = self.m(x.float())
        return x.to(input_type)

In [ ]:
def cl_init(cls, config):
    """
    Contrastive learning class init function.
    """
    cls.pooler_type = cls.model_args.pooler_type
    cls.pooler = Pooler(cls.model_args.pooler_type)
    cls.mlp = MLPLayer(config)
    cls.sim = Similarity(temp=cls.model_args.temp)
   
    # SCD - BEGIN: projector
    sizes = [cls.config.embedding_dim] + \
        list(map(int, cls.config.projector.split('-')))
    layers = []
    for i in range(len(sizes) - 2):
        if i == 0:
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=True))
        else:
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=False))
        layers.append(nn.BatchNorm1d(sizes[i + 1]))
        layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Linear(sizes[-2], sizes[-1], bias=False))
    cls.projector = nn.Sequential(*layers)
    cls.bn = nn.BatchNorm1d(sizes[-1], affine=False)
    # SCD - END: projector

    cls.init_weights()